In [1]:
import pandas as pd
import numpy as np
from scipy.stats import multivariate_normal
from gp_pref_elicit_luisa import dataset as data
from gp_pref_elicit_luisa import gaussian_process as GP
from momabs_bayesian import bayes_logistic as bayes_logistic
from gp_pref_elicit_luisa.gp_utilities import utils_user as gp_utils_users
from logistic_user import LogisticDecisionMaker as logistic 

In [2]:
# initializing dataset class
utils_comparisons = data.DatasetPairwise(num_objectives=2)
GP = GP.GPPairwise(num_objectives=2)

In [3]:
# synthetic pareto coverage set
synthetic_pcs = np.array([[0.14370116, 0.99159928],
       [0.9797389 , 0.2242916 ],
       [0.        , 1.        ],
       [0.91055917, 0.45020785],
       [0.59678925, 0.81854996],
       [1.        , 0.        ],
       [0.94198057, 0.352479  ],
       [0.81501748, 0.65358114],
       [0.99814566, 0.05429028],
       [0.33305315, 0.94955291],
       [0.28860669, 0.96123215],
       [0.99999796, 0.02591092],
       [0.98910769, 0.14092867],
       [0.18584726, 0.98334638],
       [0.05210043, 0.99838732],
       [0.87761802, 0.52114756],
       [0.74002719, 0.7144284 ],
       [0.21487083, 0.97724899],
       [0.43622937, 0.90230767],
       [0.99525346, 0.08213535]])

In [4]:
# generating random points from synthetic pcs 
start_points = synthetic_pcs[np.random.choice(synthetic_pcs.shape[0], size=2, replace=False)]
start_points

array([[0.99999796, 0.02591092],
       [0.05210043, 0.99838732]])

In [5]:
user_simulation = logistic(no_obj=2, num_features=4)
user_simulation.exact_compare(start_points[0], start_points[1])
model_space = user_simulation.sample_model()
model_space

array([ 0.24067781, -1.43027198, -0.95858984,  0.42492366])

In [6]:
thompson_point = user_simulation.thompson_sampled_point(synthetic_pcs)
thompson_point

TypeError: LogisticDecisionMaker.features() takes 1 positional argument but 2 were given

In [ ]:
user_pref = gp_utils_users.UserPreference(num_objectives=2, std_noise=0.1)

In [ ]:
# generating ground truth utility function for the starting points
ground_truth_utility_function_start = user_pref.get_preference(start_points, add_noise=True)
# getting the index of the highest utility
highest_utility_index_start = np.argmax(ground_truth_utility_function_start)
# getting the index of the lowest utility
lowest_utility_index_start = np.argmin(ground_truth_utility_function_start)
# mapping the indices of the highest utility to the actual datapoint in the dataset
highest_utility_point_dataset = start_points[highest_utility_index_start]
# mapping the indices of the lowest utility to the actual datapoint in the dataset
lowest_utility_point_dataset = start_points[lowest_utility_index_start]

# adding the highest utility point, which is the winner, to the dataset 
utils_comparisons.add_single_comparison(highest_utility_point_dataset, lowest_utility_point_dataset)


Watch out - trying to add comparison between same datapoints!


False

In [ ]:
current_best = np.max(start_points)

In [ ]:
def features(v):
    mins = []
    maxs = []
    for i in range(len(v)):
        for v_j in v[i+1]:
            mins.append(min(v[i], v_j))
            maxs.append(max(v[i], v_j))
    result = list(v) + maxs + mins
    return np.array(result)

In [ ]:
features(start_points)

TypeError: 'numpy.float64' object is not iterable

In [ ]:
y = np.array([0,1])
wprior = GP.prior_mean(x=start_points)
H = np.array([[0, 1],
              [1, 0]])


In [ ]:
# bayesian logistic regression 
bayes_logistic = bayes_logistic.fit_bayes_logistic(y=y, X=start_points, wprior=wprior, H=H, weights=None, solver='Newton-CG', bounds=None, maxiter=100)


ValueError: X must be a N*p matrix

In [ ]:
# getting the feature vectors
max_feature_vector = np.max(current_best, )
min_feature_vector = np.min(current_best, )
# combined_feature_vector = x -> 1st value vector, y -> 2nd value vector, max(x,y), min(x,y)
# eg: [1,2] -> [1,2, max(1,2), min(1,2)] = [1, 2, 2, 1]
# combined_feature_vector = [[current_best, , np.max(start_points), np.min(start_points)]]

[[array([0.14370116, 0.99159928]), 0.99159928, 0.14370116]]

In [ ]:
# multivariate normal
# multivariate_normal(mean=, cov=, allow_singular=False, seed=None)